# Hierarchical Clustering

**Hierarchical clustering** refers to a class of clustering methods that seek to build a **hierarchy** of clusters, in which some clusters contain others. In this assignment, we will explore a top-down approach, recursively bipartitioning the data using k-means.

**Note to Amazon EC2 users**: To conserve memory, make sure to stop all the other notebooks before running this notebook.

## Import packages

In [1]:
from __future__ import print_function # to conform python 2.x print to python 3.x
import graphlab
import matplotlib.pyplot as plt
import numpy as np
import sys
import os
import time
from scipy.sparse import csr_matrix
from sklearn.cluster import KMeans
from sklearn.metrics import pairwise_distances
%matplotlib inline

This non-commercial license of GraphLab Create for academic use is assigned to 1505103.sbf@ugrad.cse.buet.ac.bd and will expire on April 20, 2021.


[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: /tmp/graphlab_server_1598647117.log


## Load the Wikipedia dataset

In [2]:
wiki = graphlab.SFrame('m_4549381c276b46c6.frame_idx')

As we did in previous assignments, let's extract the TF-IDF features:

In [3]:
wiki['tf_idf'] = graphlab.text_analytics.tf_idf(wiki['text'])

To run k-means on this dataset, we should convert the data matrix into a sparse matrix.

In [4]:
def load_sparse_csr(filename):
    loader = np.load(filename)
    data = loader['data']
    indices = loader['indices']
    indptr = loader['indptr']
    shape = loader['shape']
    
    return csr_matrix( (data, indices, indptr), shape)

tf_idf = load_sparse_csr('people_wiki_tf_idf.npz')
map_index_to_word = graphlab.SFrame('m_518b98533167d279.frame_idx')

To be consistent with the k-means assignment, let's normalize all vectors to have unit norm.

In [5]:
from sklearn.preprocessing import normalize
tf_idf = normalize(tf_idf)

## Bipartition the Wikipedia dataset using k-means

Recall our workflow for clustering text data with k-means:

1. Load the dataframe containing a dataset, such as the Wikipedia text dataset.
2. Extract the data matrix from the dataframe.
3. Run k-means on the data matrix with some value of k.
4. Visualize the clustering results using the centroids, cluster assignments, and the original dataframe. We keep the original dataframe around because the data matrix does not keep auxiliary information (in the case of the text dataset, the title of each article).

Let us modify the workflow to perform bipartitioning:

1. Load the dataframe containing a dataset, such as the Wikipedia text dataset.
2. Extract the data matrix from the dataframe.
3. Run k-means on the data matrix with k=2.
4. Divide the data matrix into two parts using the cluster assignments.
5. Divide the dataframe into two parts, again using the cluster assignments. This step is necessary to allow for visualization.
6. Visualize the bipartition of data.

We'd like to be able to repeat Steps 3-6 multiple times to produce a **hierarchy** of clusters such as the following:
```
                      (root)
                         |
            +------------+-------------+
            |                          |
         Cluster                    Cluster
     +------+-----+             +------+-----+
     |            |             |            |
   Cluster     Cluster       Cluster      Cluster
```
Each **parent cluster** is bipartitioned to produce two **child clusters**. At the very top is the **root cluster**, which consists of the entire dataset.

Now we write a wrapper function to bipartition a given cluster using k-means. There are three variables that together comprise the cluster:

* `dataframe`: a subset of the original dataframe that correspond to member rows of the cluster
* `matrix`: same set of rows, stored in sparse matrix format
* `centroid`: the centroid of the cluster (not applicable for the root cluster)

Rather than passing around the three variables separately, we package them into a Python dictionary. The wrapper function takes a single dictionary (representing a parent cluster) and returns two dictionaries (representing the child clusters).

In [6]:
def bipartition(cluster, maxiter=400, num_runs=4, seed=None):
    '''cluster: should be a dictionary containing the following keys
                * dataframe: original dataframe
                * matrix:    same data, in matrix format
                * centroid:  centroid for this particular cluster'''
    
    data_matrix = cluster['matrix']
    dataframe   = cluster['dataframe']
    
    # Run k-means on the data matrix with k=2. We use scikit-learn here to simplify workflow.
    kmeans_model = KMeans(n_clusters=2, max_iter=maxiter, n_init=num_runs, random_state=seed, n_jobs=1)
    kmeans_model.fit(data_matrix)
    centroids, cluster_assignment = kmeans_model.cluster_centers_, kmeans_model.labels_
    
    # Divide the data matrix into two parts using the cluster assignments.
    data_matrix_left_child, data_matrix_right_child = data_matrix[cluster_assignment==0], \
                                                      data_matrix[cluster_assignment==1]
    
    # Divide the dataframe into two parts, again using the cluster assignments.
    cluster_assignment_sa = graphlab.SArray(cluster_assignment) # minor format conversion
    dataframe_left_child, dataframe_right_child     = dataframe[cluster_assignment_sa==0], \
                                                      dataframe[cluster_assignment_sa==1]
        
    
    # Package relevant variables for the child clusters
    cluster_left_child  = {'matrix': data_matrix_left_child,
                           'dataframe': dataframe_left_child,
                           'centroid': centroids[0]}
    cluster_right_child = {'matrix': data_matrix_right_child,
                           'dataframe': dataframe_right_child,
                           'centroid': centroids[1]}
    
    return (cluster_left_child, cluster_right_child)

The following cell performs bipartitioning of the Wikipedia dataset. Allow 2+ minutes to finish.

Note. For the purpose of the assignment, we set an explicit seed (`seed=1`) to produce identical outputs for every run. In pratical applications, you might want to use different random seeds for all runs.

In [7]:
%%time
wiki_data = {'matrix': tf_idf, 'dataframe': wiki} # no 'centroid' for the root cluster
left_child, right_child = bipartition(wiki_data, maxiter=100, num_runs=1, seed=0)

CPU times: user 2min 46s, sys: 3.3 s, total: 2min 49s
Wall time: 2min 43s


Let's examine the contents of one of the two clusters, which we call the `left_child`, referring to the tree visualization above.

In [8]:
left_child

{'centroid': array([4.73551584e-06, 0.00000000e+00, 4.54643189e-06, ...,
        1.10902519e-04, 9.02452801e-05, 2.11546300e-05]), 'dataframe': Columns:
 	URI	str
 	name	str
 	text	str
 	tf_idf	dict
 
 Rows: Unknown
 
 Data:
 +-------------------------------+-------------------------------+
 |              URI              |              name             |
 +-------------------------------+-------------------------------+
 | <http://dbpedia.org/resour... |         Alfred J. Lewy        |
 | <http://dbpedia.org/resour... |      Franz Rottensteiner      |
 | <http://dbpedia.org/resour... |         Sam Henderson         |
 | <http://dbpedia.org/resour... |        Trevor Ferguson        |
 | <http://dbpedia.org/resour... |          Cathy Caruth         |
 | <http://dbpedia.org/resour... |          Sophie Crumb         |
 | <http://dbpedia.org/resour... |         Jenn Ashworth         |
 | <http://dbpedia.org/resour... |        Jonathan Hoefler       |
 | <http://dbpedia.org/resour... | Ant

And here is the content of the other cluster we named `right_child`.

In [9]:
right_child

{'centroid': array([0.00000000e+00, 3.42095225e-06, 0.00000000e+00, ...,
        1.20929073e-04, 7.97127398e-05, 2.04603868e-05]), 'dataframe': Columns:
 	URI	str
 	name	str
 	text	str
 	tf_idf	dict
 
 Rows: Unknown
 
 Data:
 +-------------------------------+-------------------------------+
 |              URI              |              name             |
 +-------------------------------+-------------------------------+
 | <http://dbpedia.org/resour... |         Digby Morrell         |
 | <http://dbpedia.org/resour... |         Harpdog Brown         |
 | <http://dbpedia.org/resour... |             G-Enka            |
 | <http://dbpedia.org/resour... |         Aaron LaCrate         |
 | <http://dbpedia.org/resour... |          Grant Nelson         |
 | <http://dbpedia.org/resour... |         Joerg Steineck        |
 | <http://dbpedia.org/resour... | Paddy Dunne (Gaelic footba... |
 | <http://dbpedia.org/resour... |       Alexandros Mouzas       |
 | <http://dbpedia.org/resour... |    

## Visualize the bipartition

We provide you with a modified version of the visualization function from the k-means assignment. For each cluster, we print the top 5 words with highest TF-IDF weights in the centroid and display excerpts for the 8 nearest neighbors of the centroid.

In [10]:
def display_single_tf_idf_cluster(cluster, map_index_to_word):
    '''map_index_to_word: SFrame specifying the mapping betweeen words and column indices'''
    
    wiki_subset   = cluster['dataframe']
    tf_idf_subset = cluster['matrix']
    centroid      = cluster['centroid']
    
    # Print top 5 words with largest TF-IDF weights in the cluster
    idx = centroid.argsort()[::-1]
    for i in range(5):
        print('{0}:{1:.3f}'.format(map_index_to_word['category'], centroid[idx[i]])),
    print('')
    
    # Compute distances from the centroid to all data points in the cluster.
    distances = pairwise_distances(tf_idf_subset, [centroid], metric='euclidean').flatten()
    # compute nearest neighbors of the centroid within the cluster.
    nearest_neighbors = distances.argsort()
    # For 8 nearest neighbors, print the title as well as first 180 characters of text.
    # Wrap the text at 80-character mark.
    for i in range(8):
        text = ' '.join(wiki_subset[nearest_neighbors[i]]['text'].split(None, 25)[0:25])
        print('* {0:50s} {1:.5f}\n  {2:s}\n  {3:s}'.format(wiki_subset[nearest_neighbors[i]]['name'],
              distances[nearest_neighbors[i]], text[:90], text[90:180] if len(text) > 90 else ''))
    print('')

Let's visualize the two child clusters:

In [11]:
display_single_tf_idf_cluster(left_child, map_index_to_word)

['bioarchaeologist', 'leaguehockey', 'electionruss', 'teramoto', 'trumpeterpercussionist', 'spoofax', 'mendelssohni', 'crosswise', 'yec', 'asianthemed', 'masheldon', 'maywoods', 'feduring', 'seameo', '2012green', 'wrighthassell', 'lidda', 'wfo', 'ukfang', 'outfitover', 'pagbabago', 'influences1', 'stonier', 'brbbarbosa', 'ipishuna', 'researchteuvo', 'stephensens', 'titheridge', 'dunlapi', 'specs', 'komozi', 'fajita', 'sauvagein', 'brilliantmusik', 'glickenhaus', '23seat', 'selfloading', 'tankians', '333465389', 'reviewsbarasch', '89195', 'arrestedon', 'confisses', 'unclog', 'newbies', 'mcin', 'ecosocial', 'sumiswald', 'deleriums', 'trochim', 'darkenedfrom', '91ydsprofessional', 'yorkcolomina', 'athfest', 'minutetraianidis', 'radiograciette', 'textsross', 'epiculture', 'firedheading', 'redecoration', 'krywult', 'montanaform', 'kyozan', 'reformbestrebungen', 'atlantaduring', 'drumsshe', 'athttpbelenfernandezwritingsblogspotcouk', 'selingerborn', 'glycosciences', 'foundationrichardson', '

* James A. Joseph                                    0.97624
  james a joseph born 1935 is an american former diplomatjoseph is professor of the practice
   of public policy studies at duke university and founder of



In [12]:
display_single_tf_idf_cluster(right_child, map_index_to_word)

['bioarchaeologist', 'leaguehockey', 'electionruss', 'teramoto', 'trumpeterpercussionist', 'spoofax', 'mendelssohni', 'crosswise', 'yec', 'asianthemed', 'masheldon', 'maywoods', 'feduring', 'seameo', '2012green', 'wrighthassell', 'lidda', 'wfo', 'ukfang', 'outfitover', 'pagbabago', 'influences1', 'stonier', 'brbbarbosa', 'ipishuna', 'researchteuvo', 'stephensens', 'titheridge', 'dunlapi', 'specs', 'komozi', 'fajita', 'sauvagein', 'brilliantmusik', 'glickenhaus', '23seat', 'selfloading', 'tankians', '333465389', 'reviewsbarasch', '89195', 'arrestedon', 'confisses', 'unclog', 'newbies', 'mcin', 'ecosocial', 'sumiswald', 'deleriums', 'trochim', 'darkenedfrom', '91ydsprofessional', 'yorkcolomina', 'athfest', 'minutetraianidis', 'radiograciette', 'textsross', 'epiculture', 'firedheading', 'redecoration', 'krywult', 'montanaform', 'kyozan', 'reformbestrebungen', 'atlantaduring', 'drumsshe', 'athttpbelenfernandezwritingsblogspotcouk', 'selingerborn', 'glycosciences', 'foundationrichardson', '

* Kayla Bashore Smedley                              0.97496
  kayla bashore born february 20 1983 in daegu south korea is an american field hockey defen
  der and midfielder now living in san diego california she
* Cher                                               0.97510
  cher r born cherilyn sarkisian may 20 1946 is an american singer actress and television ho
  st described as embodying female autonomy in a maledominated industry



The right cluster consists of athletes and artists (singers and actors/actresses), whereas the left cluster consists of non-athletes and non-artists. So far, we have a single-level hierarchy consisting of two clusters, as follows:

```
                                           Wikipedia
                                               +
                                               |
                    +--------------------------+--------------------+
                    |                                               |
                    +                                               +
         Non-athletes/artists                                Athletes/artists
```

Is this hierarchy good enough? **When building a hierarchy of clusters, we must keep our particular application in mind.** For instance, we might want to build a **directory** for Wikipedia articles. A good directory would let you quickly narrow down your search to a small set of related articles. The categories of athletes and non-athletes are too general to facilitate efficient search. For this reason, we decide to build another level into our hierarchy of clusters with the goal of getting more specific cluster structure at the lower level. To that end, we subdivide both the `athletes/artists` and `non-athletes/artists` clusters.

## Perform recursive bipartitioning

### Cluster of athletes and artists

To help identify the clusters we've built so far, let's give them easy-to-read aliases:

In [12]:
non_athletes_artists   = left_child
athletes_artists       = right_child

Using the bipartition function, we produce two child clusters of the athlete cluster:

In [14]:
# Bipartition the cluster of athletes and artists
left_child_athletes_artists, right_child_athletes_artists = bipartition(athletes_artists, maxiter=100, num_runs=6, seed=1)

The left child cluster mainly consists of athletes:

In [15]:
display_single_tf_idf_cluster(left_child_athletes_artists, map_index_to_word)

['bioarchaeologist', 'leaguehockey', 'electionruss', 'teramoto', 'trumpeterpercussionist', 'spoofax', 'mendelssohni', 'crosswise', 'yec', 'asianthemed', 'masheldon', 'maywoods', 'feduring', 'seameo', '2012green', 'wrighthassell', 'lidda', 'wfo', 'ukfang', 'outfitover', 'pagbabago', 'influences1', 'stonier', 'brbbarbosa', 'ipishuna', 'researchteuvo', 'stephensens', 'titheridge', 'dunlapi', 'specs', 'komozi', 'fajita', 'sauvagein', 'brilliantmusik', 'glickenhaus', '23seat', 'selfloading', 'tankians', '333465389', 'reviewsbarasch', '89195', 'arrestedon', 'confisses', 'unclog', 'newbies', 'mcin', 'ecosocial', 'sumiswald', 'deleriums', 'trochim', 'darkenedfrom', '91ydsprofessional', 'yorkcolomina', 'athfest', 'minutetraianidis', 'radiograciette', 'textsross', 'epiculture', 'firedheading', 'redecoration', 'krywult', 'montanaform', 'kyozan', 'reformbestrebungen', 'atlantaduring', 'drumsshe', 'athttpbelenfernandezwritingsblogspotcouk', 'selingerborn', 'glycosciences', 'foundationrichardson', '

* Tommy Anderson (footballer)                        0.96060
  thomas cowan tommy anderson born 24 september 1934 in haddington is a scottish former prof
  essional footballer he played as a forward and was noted for



On the other hand, the right child cluster consists mainly of artists (singers and actors/actresses):

In [16]:
display_single_tf_idf_cluster(right_child_athletes_artists, map_index_to_word)

['bioarchaeologist', 'leaguehockey', 'electionruss', 'teramoto', 'trumpeterpercussionist', 'spoofax', 'mendelssohni', 'crosswise', 'yec', 'asianthemed', 'masheldon', 'maywoods', 'feduring', 'seameo', '2012green', 'wrighthassell', 'lidda', 'wfo', 'ukfang', 'outfitover', 'pagbabago', 'influences1', 'stonier', 'brbbarbosa', 'ipishuna', 'researchteuvo', 'stephensens', 'titheridge', 'dunlapi', 'specs', 'komozi', 'fajita', 'sauvagein', 'brilliantmusik', 'glickenhaus', '23seat', 'selfloading', 'tankians', '333465389', 'reviewsbarasch', '89195', 'arrestedon', 'confisses', 'unclog', 'newbies', 'mcin', 'ecosocial', 'sumiswald', 'deleriums', 'trochim', 'darkenedfrom', '91ydsprofessional', 'yorkcolomina', 'athfest', 'minutetraianidis', 'radiograciette', 'textsross', 'epiculture', 'firedheading', 'redecoration', 'krywult', 'montanaform', 'kyozan', 'reformbestrebungen', 'atlantaduring', 'drumsshe', 'athttpbelenfernandezwritingsblogspotcouk', 'selingerborn', 'glycosciences', 'foundationrichardson', '

* Natashia Williams                                  0.96715
  natashia williamsblach born august 2 1978 is an american actress and former wonderbra camp
  aign model who is perhaps best known for her role as shane



Our hierarchy of clusters now looks like this:
```
                                           Wikipedia
                                               +
                                               |
                    +--------------------------+--------------------+
                    |                                               |
                    +                                               +
         Non-athletes/artists                                Athletes/artists
                                                                    +
                                                                    |
                                                         +----------+----------+
                                                         |                     |
                                                         |                     |
                                                         +                     |
                                                     athletes               artists
```

Should we keep subdividing the clusters? If so, which cluster should we subdivide? To answer this question, we again think about our application. Since we organize our directory by topics, it would be nice to have topics that are about as coarse as each other. For instance, if one cluster is about baseball, we expect some other clusters about football, basketball, volleyball, and so forth. That is, **we would like to achieve similar level of granularity for all clusters.**

Both the athletes and artists node can be subdivided more, as each one can be divided into more descriptive professions (singer/actress/painter/director, or baseball/football/basketball, etc.). Let's explore subdividing the athletes cluster further to produce finer child clusters.

Let's give the clusters aliases as well:

In [17]:
athletes    = left_child_athletes_artists
artists     = right_child_athletes_artists

### Cluster of athletes

In answering the following quiz question, take a look at the topics represented in the top documents (those closest to the centroid), as well as the list of words with highest TF-IDF weights.

Let us bipartition the cluster of athletes.

In [18]:
left_child_athletes, right_child_athletes = bipartition(athletes, maxiter=100, num_runs=6, seed=1)

In [19]:
display_single_tf_idf_cluster(left_child_athletes, map_index_to_word)
display_single_tf_idf_cluster(right_child_athletes, map_index_to_word)

['bioarchaeologist', 'leaguehockey', 'electionruss', 'teramoto', 'trumpeterpercussionist', 'spoofax', 'mendelssohni', 'crosswise', 'yec', 'asianthemed', 'masheldon', 'maywoods', 'feduring', 'seameo', '2012green', 'wrighthassell', 'lidda', 'wfo', 'ukfang', 'outfitover', 'pagbabago', 'influences1', 'stonier', 'brbbarbosa', 'ipishuna', 'researchteuvo', 'stephensens', 'titheridge', 'dunlapi', 'specs', 'komozi', 'fajita', 'sauvagein', 'brilliantmusik', 'glickenhaus', '23seat', 'selfloading', 'tankians', '333465389', 'reviewsbarasch', '89195', 'arrestedon', 'confisses', 'unclog', 'newbies', 'mcin', 'ecosocial', 'sumiswald', 'deleriums', 'trochim', 'darkenedfrom', '91ydsprofessional', 'yorkcolomina', 'athfest', 'minutetraianidis', 'radiograciette', 'textsross', 'epiculture', 'firedheading', 'redecoration', 'krywult', 'montanaform', 'kyozan', 'reformbestrebungen', 'atlantaduring', 'drumsshe', 'athttpbelenfernandezwritingsblogspotcouk', 'selingerborn', 'glycosciences', 'foundationrichardson', '

* Gord Sherven                                       0.95829
  gordon r sherven born august 21 1963 in gravelbourg saskatchewan and raised in mankota sas
  katchewan is a retired canadian professional ice hockey forward who played
* Ashley Prescott                                    0.95974
  ashley prescott born 11 september 1972 is a former australian rules footballer he played w
  ith the richmond and fremantle football clubs in the afl between
* Jason Roberts (footballer)                         0.95975
  jason andre davis roberts mbe born 25 january 1978 is a former professional footballer and
   now a football punditborn in park royal london roberts was
* Chris Day                                          0.95977
  christopher nicholas chris day born 28 july 1975 is an english professional footballer who
   plays as a goalkeeper for stevenageday started his career at tottenham
* Sol Campbell                                       0.96051
  sulzeer jeremiah sol campbell born 18 sep

**Quiz Question**. Which diagram best describes the hierarchy right after splitting the `athletes` cluster? Refer to the quiz form for the diagrams.

**Caution**. The granularity criteria is an imperfect heuristic and must be taken with a grain of salt. It takes a lot of manual intervention to obtain a good hierarchy of clusters.

* **If a cluster is highly mixed, the top articles and words may not convey the full picture of the cluster.** Thus, we may be misled if we judge the purity of clusters solely by their top documents and words. 
* **Many interesting topics are hidden somewhere inside the clusters but do not appear in the visualization.** We may need to subdivide further to discover new topics. For instance, subdividing the `ice_hockey_football` cluster led to the appearance of runners and golfers.

### Cluster of non-athletes

Now let us subdivide the cluster of non-athletes.

In [13]:
%%time 
# Bipartition the cluster of non-athletes
left_child_non_athletes_artists, right_child_non_athletes_artists = bipartition(non_athletes_artists, maxiter=100, num_runs=3, seed=1)

CPU times: user 10min 22s, sys: 23 s, total: 10min 45s
Wall time: 9min 49s


In [14]:
display_single_tf_idf_cluster(left_child_non_athletes_artists, map_index_to_word)

['bioarchaeologist', 'leaguehockey', 'electionruss', 'teramoto', 'trumpeterpercussionist', 'spoofax', 'mendelssohni', 'crosswise', 'yec', 'asianthemed', 'masheldon', 'maywoods', 'feduring', 'seameo', '2012green', 'wrighthassell', 'lidda', 'wfo', 'ukfang', 'outfitover', 'pagbabago', 'influences1', 'stonier', 'brbbarbosa', 'ipishuna', 'researchteuvo', 'stephensens', 'titheridge', 'dunlapi', 'specs', 'komozi', 'fajita', 'sauvagein', 'brilliantmusik', 'glickenhaus', '23seat', 'selfloading', 'tankians', '333465389', 'reviewsbarasch', '89195', 'arrestedon', 'confisses', 'unclog', 'newbies', 'mcin', 'ecosocial', 'sumiswald', 'deleriums', 'trochim', 'darkenedfrom', '91ydsprofessional', 'yorkcolomina', 'athfest', 'minutetraianidis', 'radiograciette', 'textsross', 'epiculture', 'firedheading', 'redecoration', 'krywult', 'montanaform', 'kyozan', 'reformbestrebungen', 'atlantaduring', 'drumsshe', 'athttpbelenfernandezwritingsblogspotcouk', 'selingerborn', 'glycosciences', 'foundationrichardson', '

* Archie Brown                                       0.97691
  archibald haworth brown cmg fba commonly known as archie brown born 10 may 1938 is a briti
  sh political scientist and historian in 2005 he became



In [15]:
display_single_tf_idf_cluster(right_child_non_athletes_artists, map_index_to_word)

['bioarchaeologist', 'leaguehockey', 'electionruss', 'teramoto', 'trumpeterpercussionist', 'spoofax', 'mendelssohni', 'crosswise', 'yec', 'asianthemed', 'masheldon', 'maywoods', 'feduring', 'seameo', '2012green', 'wrighthassell', 'lidda', 'wfo', 'ukfang', 'outfitover', 'pagbabago', 'influences1', 'stonier', 'brbbarbosa', 'ipishuna', 'researchteuvo', 'stephensens', 'titheridge', 'dunlapi', 'specs', 'komozi', 'fajita', 'sauvagein', 'brilliantmusik', 'glickenhaus', '23seat', 'selfloading', 'tankians', '333465389', 'reviewsbarasch', '89195', 'arrestedon', 'confisses', 'unclog', 'newbies', 'mcin', 'ecosocial', 'sumiswald', 'deleriums', 'trochim', 'darkenedfrom', '91ydsprofessional', 'yorkcolomina', 'athfest', 'minutetraianidis', 'radiograciette', 'textsross', 'epiculture', 'firedheading', 'redecoration', 'krywult', 'montanaform', 'kyozan', 'reformbestrebungen', 'atlantaduring', 'drumsshe', 'athttpbelenfernandezwritingsblogspotcouk', 'selingerborn', 'glycosciences', 'foundationrichardson', '

   is a member of the security intelligence review committee which
* Liz Cunningham                                     0.96350
  elizabeth anne liz cunningham is an australian politician she was an independent member of
   the legislative assembly of queensland from 1995 to 2015 representing the
* Doug Lewis                                         0.96353
  douglas grinslade doug lewis pc qc born april 17 1938 is a former canadian politician a ch
  artered accountant and lawyer by training lewis entered the
* David Anderson (British Columbia politician)       0.96379
  david a anderson pc oc born august 16 1937 in victoria british columbia is a former canadi
  an cabinet minister educated at victoria college in victoria



The clusters are not as clear, but the left cluster has a tendency to show important female figures, and the right one to show politicians and government officials.

Let's divide them further.

In [16]:
female_figures = left_child_non_athletes_artists
politicians_etc = right_child_non_athletes_artists

**Quiz Question**. Let us bipartition the clusters `female_figures` and `politicians`. Which diagram best describes the resulting hierarchy of clusters for the non-athletes? Refer to the quiz for the diagrams.

**Note**. Use `maxiter=100, num_runs=6, seed=1` for consistency of output.

In [17]:
left_child_female, right_child_female = bipartition(female_figures, maxiter=100, num_runs=6, seed=1)
display_single_tf_idf_cluster(left_child_female, map_index_to_word)
display_single_tf_idf_cluster(right_child_female, map_index_to_word)

['bioarchaeologist', 'leaguehockey', 'electionruss', 'teramoto', 'trumpeterpercussionist', 'spoofax', 'mendelssohni', 'crosswise', 'yec', 'asianthemed', 'masheldon', 'maywoods', 'feduring', 'seameo', '2012green', 'wrighthassell', 'lidda', 'wfo', 'ukfang', 'outfitover', 'pagbabago', 'influences1', 'stonier', 'brbbarbosa', 'ipishuna', 'researchteuvo', 'stephensens', 'titheridge', 'dunlapi', 'specs', 'komozi', 'fajita', 'sauvagein', 'brilliantmusik', 'glickenhaus', '23seat', 'selfloading', 'tankians', '333465389', 'reviewsbarasch', '89195', 'arrestedon', 'confisses', 'unclog', 'newbies', 'mcin', 'ecosocial', 'sumiswald', 'deleriums', 'trochim', 'darkenedfrom', '91ydsprofessional', 'yorkcolomina', 'athfest', 'minutetraianidis', 'radiograciette', 'textsross', 'epiculture', 'firedheading', 'redecoration', 'krywult', 'montanaform', 'kyozan', 'reformbestrebungen', 'atlantaduring', 'drumsshe', 'athttpbelenfernandezwritingsblogspotcouk', 'selingerborn', 'glycosciences', 'foundationrichardson', '

* Elijah Anderson                                    0.97857
  elijah anderson is an american sociologist he holds the william k lanman jr professorship 
  in sociology at yale university where he teaches and directs the

['bioarchaeologist', 'leaguehockey', 'electionruss', 'teramoto', 'trumpeterpercussionist', 'spoofax', 'mendelssohni', 'crosswise', 'yec', 'asianthemed', 'masheldon', 'maywoods', 'feduring', 'seameo', '2012green', 'wrighthassell', 'lidda', 'wfo', 'ukfang', 'outfitover', 'pagbabago', 'influences1', 'stonier', 'brbbarbosa', 'ipishuna', 'researchteuvo', 'stephensens', 'titheridge', 'dunlapi', 'specs', 'komozi', 'fajita', 'sauvagein', 'brilliantmusik', 'glickenhaus', '23seat', 'selfloading', 'tankians', '333465389', 'reviewsbarasch', '89195', 'arrestedon', 'confisses', 'unclog', 'newbies', 'mcin', 'ecosocial', 'sumiswald', 'deleriums', 'trochim', 'darkenedfrom', '91ydsprofessional', 'yorkcolomina', 'athfest', 'minutetraianidis', 'radiograciette', 'textsross', 'epiculture',

* Barbara T. Smith                                   0.95126
  barbara turner smith born 1931 pasadena california is an american artist known for her per
  formance work in the late 1960s she studied painting art history



In [18]:
left_child_politicians, right_child_politicians = bipartition(politicians_etc, maxiter=100, num_runs=6, seed=1)
display_single_tf_idf_cluster(left_child_politicians, map_index_to_word)
display_single_tf_idf_cluster(right_child_politicians, map_index_to_word)

['bioarchaeologist', 'leaguehockey', 'electionruss', 'teramoto', 'trumpeterpercussionist', 'spoofax', 'mendelssohni', 'crosswise', 'yec', 'asianthemed', 'masheldon', 'maywoods', 'feduring', 'seameo', '2012green', 'wrighthassell', 'lidda', 'wfo', 'ukfang', 'outfitover', 'pagbabago', 'influences1', 'stonier', 'brbbarbosa', 'ipishuna', 'researchteuvo', 'stephensens', 'titheridge', 'dunlapi', 'specs', 'komozi', 'fajita', 'sauvagein', 'brilliantmusik', 'glickenhaus', '23seat', 'selfloading', 'tankians', '333465389', 'reviewsbarasch', '89195', 'arrestedon', 'confisses', 'unclog', 'newbies', 'mcin', 'ecosocial', 'sumiswald', 'deleriums', 'trochim', 'darkenedfrom', '91ydsprofessional', 'yorkcolomina', 'athfest', 'minutetraianidis', 'radiograciette', 'textsross', 'epiculture', 'firedheading', 'redecoration', 'krywult', 'montanaform', 'kyozan', 'reformbestrebungen', 'atlantaduring', 'drumsshe', 'athttpbelenfernandezwritingsblogspotcouk', 'selingerborn', 'glycosciences', 'foundationrichardson', '

* Pasco Bowman II                                    0.91907
  pasco middleton bowman ii born 1933 is a senior federal judge on the united states court o
  f appeals for the eighth circuit a former fulbright

['bioarchaeologist', 'leaguehockey', 'electionruss', 'teramoto', 'trumpeterpercussionist', 'spoofax', 'mendelssohni', 'crosswise', 'yec', 'asianthemed', 'masheldon', 'maywoods', 'feduring', 'seameo', '2012green', 'wrighthassell', 'lidda', 'wfo', 'ukfang', 'outfitover', 'pagbabago', 'influences1', 'stonier', 'brbbarbosa', 'ipishuna', 'researchteuvo', 'stephensens', 'titheridge', 'dunlapi', 'specs', 'komozi', 'fajita', 'sauvagein', 'brilliantmusik', 'glickenhaus', '23seat', 'selfloading', 'tankians', '333465389', 'reviewsbarasch', '89195', 'arrestedon', 'confisses', 'unclog', 'newbies', 'mcin', 'ecosocial', 'sumiswald', 'deleriums', 'trochim', 'darkenedfrom', '91ydsprofessional', 'yorkcolomina', 'athfest', 'minutetraianidis', 'radiograciette', 'textsross', 'epiculture', 'firedheadin

* Carol Skelton                                      0.95928
  carol skelton pc born december 12 1945 in biggar saskatchewan is a canadian politician she
   is a member of the security intelligence review committee which
* Margaret Beckett                                   0.95972
  dame margaret mary beckett dbe mp born 15 january 1943 is a british labour party politicia
  n who has been the member of parliament mp for
* Liz Cunningham                                     0.96009
  elizabeth anne liz cunningham is an australian politician she was an independent member of
   the legislative assembly of queensland from 1995 to 2015 representing the
* Monique Landry                                     0.96012
  monique landry pc born december 25 1937 is a former canadian politician a physiotherapist 
  and administrator she was first elected to the canadian house of

